In [1]:
import os
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [2]:
from graphmodels.graphmodel import GraphModel, concatenate_graph_specs
from graphmodels.validation import score_model, plot_diagnostic, plot_baseline_vs_computation
import pandas as pd
import plotly.express as px
from sklearn.metrics import r2_score
from graphmodels.database import select_dataset
from models.Hermen.model_GE3 import GE3_models

from graphmodels.utils import df_to_data_dict, get_X_y_from_data

# Newer

In [3]:
pop = select_dataset('population')
#emissions = select_dataset('emissions')

In [4]:
emissions = pd.read_csv('models/Hermen/data/GY_GE_GU_GM_GP_GT.csv')

/home/simon/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
emissions.query("Variable == 'TMi' and ISO == 'HUN' and Year == 2017")

,ISO,Item,Description,Year,Unit,Value,Variable,FBS_item
3096654,HUN,Asses,Amount excreted in manure (N content),2017,kg,5.694000e+04,TMi,"Meat, Other"
3096712,HUN,"Cattle, dairy",Amount excreted in manure (N content),2017,kg,1.714405e+07,TMi,Milk - Excluding Butter
3096770,HUN,"Cattle, non-dairy",Amount excreted in manure (N content),2017,kg,3.036975e+07,TMi,Bovine Meat
3096828,HUN,"Chickens, broilers",Amount excreted in manure (N content),2017,kg,7.439112e+06,TMi,Poultry Meat
3096886,HUN,"Chickens, layers",Amount excreted in manure (N content),2017,kg,6.163186e+06,TMi,Eggs
3096944,HUN,Ducks,Amount excreted in manure (N content),2017,kg,3.298853e+06,TMi,Poultry Meat
3097002,HUN,Goats,Amount excreted in manure (N content),2017,kg,1.456963e+06,TMi,Mutton & Goat Meat
3097060,HUN,Horses,Amount excreted in manure (N content),2017,kg,2.187920e+06,TMi,"Meat, Other"
3097118,HUN,Mules,Amount excreted in manure (N content),2017,kg,2.519595e+03,TMi,"Meat, Other"
3097176,HUN,Sheep,Amount excreted in manure (N content),2017,kg,1.817870e+07,TMi,Mutton & Goat Meat


In [6]:
model_df = pd.concat([emissions, pop])
model_df = model_df[~model_df.Item.isin(['All Animals'])]

In [7]:
test = model_df.query('Year > 2017').drop(columns=['Description', 'Unit', 'FBS_item'])
test.to_csv('outputs/GE3_df.csv')

In [8]:
from graphmodels.utils import df_to_data_dict

def get_X_y_from_data(model, data_dict):
    '''TO CLEAN UP'''
    X = {key: data_dict[key] for key in model.inputs_() + model.parameters_()}
    y = {key: data_dict[key] for key in model.variables_() + model.outputs_() if key in data_dict}
    return X, y


def df_to_dict(df):
    X = {}
    for code in df.columns:
        X[code] = df[code]#.fillna(0)
    return X


def fill_missing_values(df):
    return df
#     return df.groupby(level='ISO').fillna(method='ffill')\
#              .groupby(level='ISO').fillna(method='bfill')


def df_to_data_dict(df, itemized):
    data_dict = {}

    non_item_df = df[~df.Variable.isin(itemized)].pivot(
        index=['ISO', 'Year'], columns='Variable', values='Value')
    non_item_df = fill_missing_values(non_item_df)
    item_df = df[df.Variable.isin(itemized)].pivot(
        index=['ISO', 'Year', 'Item'], columns='Variable', values='Value')

    data_dict.update(df_to_dict(non_item_df))
    data_dict.update(df_to_dict(item_df))

    return data_dict


In [9]:
itemized = ['TAi', 'EF_EEi', 'EECH4', 'TEE_CO2eq',
       'M_ASi', 'EF_ASi', 'E_ASi', 'TMA_CO2eq', 'M_Ti', 'EF_CH4Ti',
       'EF_Ti', 'E_TCH4i', 'E_Ti', 'TMT_CO2eq', 'M_LPi', 'EF_Li', 'E_Li',
       'TMP_CO2eq', 'TMi', 'MM_ASi', 'MM_LPi', 'MM_Ti', 'MYi',]

data_dict = df_to_data_dict(test, itemized)

In [10]:
res = []
for model_name, model in GE3_models.items():
    print(model_name)
    X, y = get_X_y_from_data(model, data_dict)
    print(score_model(model, X, y))
    print()

TMi_model
TMi
    r2  correlation          rmse Variable
0  1.0          1.0  4.024862e-16      TMi

M_xi_model
M_Ti
M_LPi
M_ASi
    r2  correlation          rmse Variable
0  1.0          1.0  5.281617e-16     M_Ti
1  1.0          1.0  2.256885e-16    M_LPi
2  1.0          1.0  3.966192e-16    M_ASi

TMP_CO2eq_model
TMP_CO2eq
         r2  correlation       rmse   Variable
0  0.999997     0.999999  24.610266  TMP_CO2eq

TMT_CO2eq_model
E_Ti
E_TCH4i
TMT_CO2eq
         r2  correlation          rmse   Variable
0  0.999992     0.999997  4.015280e-05       E_Ti
1  1.000000     1.000000  2.367571e-08    E_TCH4i
2  0.999997     0.999999  3.858853e+00  TMT_CO2eq

TMA_CO2eq_model
TMA_CO2eq
         r2  correlation      rmse   Variable
0  0.999993          1.0  1.897086  TMA_CO2eq

TEE_CO2eq_model
TEE_CO2eq
    r2  correlation          rmse   Variable
0  1.0          1.0  7.442376e-10  TEE_CO2eq

FE_CO2eq_model
FE_CO2eq
         r2  correlation         rmse  Variable
0  0.999991     0.999999  381

In [11]:
X, y = get_X_y_from_data(GE3_models['GE3_model'], data_dict)
res = GE3_models['GE3_model'].run(X)

In [12]:
data_dict['MM_LPi'].loc['HUN', 2018] + data_dict['MM_Ti'].loc['HUN', 2018] #+ data_dict['MM_ASi'].loc['HUN', 2017] 

Item
Asses                 1.0
Cattle                1.0
Cattle, dairy         1.0
Cattle, non-dairy     1.0
Chickens              1.0
Chickens, broilers    1.0
Chickens, layers      1.0
Ducks                 1.0
Goats                 1.0
Horses                1.0
Mules                 1.0
Mules and Asses       1.0
Poultry Birds         1.0
Sheep                 1.0
Sheep and Goats       1.0
Swine                 1.0
Swine, breeding       1.0
Swine, market         1.0
Turkeys               1.0
dtype: float64

In [13]:
def run_GE3_scenario(data_dict, MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3):

    data_dict = data_dict.copy()

    data_dict['MM_Ti'] = MM_Ti
    data_dict['MM_ASi'] = MM_ASi
    data_dict['MM_LPi'] = MM_LPi

    results = GE3_models['GE3_model'].run(data_dict)

    return results

In [14]:
filtered_data_dict = {k: v.loc['FRA', 2018, :] for k, v in data_dict.items()}

In [15]:
result =  run_GE3_scenario(filtered_data_dict, 1/3, 1/3, 1/3)

In [16]:
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
test = {k: v for k, v in result.items() if k in ['TEE_CO2eq', 'TMA_CO2eq', 'TMT_CO2eq', 'TMP_CO2eq']}
def format_data_dict_sankey(data_dict=test):
    data_dict = data_dict.copy()

    # grab data and format
    data = pd.concat([v.to_frame(name='Value').assign(Variable=k) for k, v in data_dict.items()], axis=0).reset_index().dropna()
    
    data = pd.concat([data, data.groupby('Variable').sum().reset_index().rename(columns={"Variable": 'Item'}).assign(Variable='Non-CO2 agricultural emissions')])

    # add encoding for Sanky
    le = LabelEncoder() 
    encoded = le.fit_transform(data[['Item', 'Variable']].values.flatten()).reshape(-1, 2)

    data[['Source', 'Target']] = encoded

    return data,  le.classes_

def plot_sanky_GE3(data, classes):
    '''To improve, should write a more general wrapper for Sankey'''

    fig = go.Figure(data=[go.Sankey(
        node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = classes,
        #color = "blue"
        ),
        link = dict(
        target = data['Target'],
        source= data['Source'],
        value = data['Value']
    ))])

    fig.update_layout(title_text=f"Agricultural Animal Emissions", font_size=10)
    return fig


d, c = format_data_dict_sankey()


fig = plot_sanky_GE3(d, c)

In [18]:
fig